# requeset 库 探索
>开发哲学
>Requests 是以 PEP 20 的箴言为中心开发的

1. Beautiful is better than ugly.(美丽优于丑陋)
2. Explicit is better than implicit.(直白优于含蓄)
3. Simple is better than complex.(简单优于复杂)
4. Complex is better than complicated.(复杂优于繁琐)
5. Readability counts.(可读性很重要)  
对于 Requests 所有的贡献都应牢记这些重要的准则。

## 安装
### pip
`pip install requesets`

### git socure code
`git clone git://github.com/kennethreitz/requests.git`

之后

`cd requeset
pip install`


## 发送请求

In [1]:
#导入Request 模块
import requests

#然后，尝试获取某个网页。本例子中，我们来获取 Github 的公共时间线：
r =  requests.get('https://api.github.com/events')
#现在，我们有一个名为 r 的 Response 对象。我们可以从这个对象中获取所有我们想要的信息。

#Requests 简便的 API 意味着所有 HTTP 请求类型都是显而易见的。
#例如，你可以这样发送一个 HTTP POST 请求：
r = requests.post('http://httpbin.org/post', data = {'key':'value'})

#漂亮，对吧？那么其他 HTTP 请求类型：PUT，DELETE，HEAD 以及 OPTIONS 又是如何的呢？
#都是一样的简单：
r = requests.put('http://httpbin.org/put', data = {'key':'value'})
r = requests.delete('http://httpbin.org/delete')
r = requests.head('http://httpbin.org/get')
r = requests.options('http://httpbin.org/get')


## 传递URL参数

In [3]:
#你也许经常想为 URL 的查询字符串(query string)传递某种数据。如果你是手工构建 URL，那么数据会以键/值对的形式置于 URL 中，
#跟在一个问号的后面。例如， httpbin.org/get?key=val。 
#Requests 允许你使用 params 关键字参数，以一个字符串字典来提供这些参数。
#举例来说，如果你想传递 key1=value1 和 key2=value2 到 httpbin.org/get ，
#那么你可以使用如下代码：

payload = {'key1':'value1','key2':'value2'}
r = requests.get("http://httpbin.org/get",params=payload)

print(r.url)

#注意字典里值为 None 的键都不会被添加到 URL 的查询字符串里。

#你还可以将一个列表作为值传入：

payload = {'key1': 'value1', 'key2': ['value2', 'value3']}

r = requests.get('http://httpbin.org/get', params=payload)
print(r.url)



http://httpbin.org/get?key1=value1&key2=value2
http://httpbin.org/get?key1=value1&key2=value2&key2=value3


## 响应内容
我们能读取服务器响应的内容。再次以 GitHub 时间线为例：

In [8]:
import requests
r = requests.get('https://api.github.com/events')

#r.text

#Requests 会自动解码来自服务器的内容。大多数 unicode 字符集都能被无缝地解码。

#请求发出后，Requests 会基于 HTTP 头部对响应的编码作出有根据的推测。
#当你访问 r.text 之时，Requests 会使用其推测的文本编码。
#你可以找出 Requests 使用了什么编码，并且能够使用 r.encoding 属性来改变它：
r.encoding
#r.encoding='ISO-8859-1'

'utf-8'

如果你改变了编码，每当你访问 r.text ，Request 都将会使用 r.encoding 的新值。你可能希望在使用特殊逻辑计算出文本的编码的情况下来修改编码。比如 HTTP 和 XML 自身可以指定编码。这样的话，你应该使用 r.content 来找到编码，然后设置 r.encoding 为相应的编码。这样就能使用正确的编码解析 r.text 了。

在你需要的情况下，Requests 也可以使用定制的编码。如果你创建了自己的编码，并使用 codecs 模块进行注册，你就可以轻松地使用这个解码器名称作为 r.encoding 的值， 然后由 Requests 来为你处理编码。

## 二进制响应内容
你也能以字节的方式访问请求响应体，对于非文本请求：

In [23]:
pic = requests.get('http://www.yjccity.com/wp-content/uploads/2017/02/201702200621282.jpg')

#pic.content

from PIL import Image
from io import BytesIO

i = Image.open(BytesIO(pic.content))

print(i.show())


None


## JSON 响应内容
Requests 中也有一个内置的 JSON 解码器，助你处理 JSON 数据

In [ ]:
import requests

r = requests.get('https://api.github.com/events')
r.json()

如果 JSON 解码失败， r.json() 就会抛出一个异常。例如，响应内容是 401 (Unauthorized)，尝试访问 r.json() 将会抛出 ValueError: No JSON object could be decoded 异常。

需要注意的是，成功调用 r.json() 并**不**意味着响应的成功。有的服务器会在失败的响应中包含一个 JSON 对象（比如 HTTP 500 的错误细节）。这种 JSON 会被解码返回。要检查请求是否成功，请使用 r.raise_for_status() 或者检查 r.status_code 是否和你的期望相同。

## 原始响应内容
在罕见的情况下，你可能想获取来自服务器的原始套接字响应，那么你可以访问 r.raw。 如果你确实想这么干，那请你确保在初始请求中设置了 stream=True。具体你可以这么做：

In [21]:
r = requests.get('https://api.github.com/events', stream=True)

#r.raw

r.raw.read(10)

#但一般情况下，你应该以下面的模式将文本流保存到文件：
'''
with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size):
        fd.write(chunk)
'''


"\nwith open(filename, 'wb') as fd:\n    for chunk in r.iter_content(chunk_size):\n        fd.write(chunk)\n"

## 定制请求头
如果你想为请求添加 HTTP 头部，只要简单地传递一个 dict 给 headers 参数就可以了。

例如，在前一个示例中我们没有指定 content-type:

In [24]:
url = 'https://api.github.com/some/endpoint'

headers = {'user-agent': 'my-app/0.0.1'}

r = requests.get(url, headers=headers)

注意: 定制 header 的优先级低于某些特定的信息源，例如：

如果在 .netrc 中设置了用户认证信息，使用 headers= 设置的授权就不会生效。而如果设置了 auth= 参数，``.netrc`` 的设置就无效了。
如果被重定向到别的主机，授权 header 就会被删除。
代理授权 header 会被 URL 中提供的代理身份覆盖掉。
在我们能判断内容长度的情况下，header 的 Content-Length 会被改写。
更进一步讲，Requests 不会基于定制 header 的具体情况改变自己的行为。只不过在最后的请求中，所有的 header 信息都会被传递进去。

注意: 所有的 header 值必须是 string、bytestring 或者 unicode。尽管传递 unicode header 也是允许的，但不建议这样做。

## 更加复杂的 POST 请求
通常，你想要发送一些编码为表单形式的数据——非常像一个 HTML 表单。要实现这个，只需简单地传递一个字典给 data 参数。你的数据字典在发出请求时会自动编码为表单形式

In [ ]:
payload = {'key1': 'value1', 'key2': 'value2'}

r = requests.post("http://httpbin.org/post", data=payload)

#print(r.text)

#你还可以为 data 参数传入一个元组列表。
#在表单中多个元素使用同一 key 的时候，这种方式尤其有效：

payload = (('key1', 'value1'), ('key1', 'value2'))

r = requests.post('http://httpbin.org/post', data=payload)

print(r.text)

## 响应状态码
我们可以检测响应状态码：

In [28]:
r = requests.get('http://httpbin.org/get')

r.status_code

#为方便引用，Requests还附带了一个内置的状态码查询对象
r.status_code == requests.codes.ok

#如果发送了一个错误请求(一个 4XX 客户端错误，或者 5XX 服务器错误响应)，
#我们可以通过 Response.raise_for_status() 来抛出异常：

bad_r = requests.get('http://httpbin.org/status/404')
bad_r.status_code

bad_r.raise_for_status()

#但是，由于我们的例子中 r 的 status_code 是 200 ，
#当我们调用 raise_for_status() 时，得到的是：
r.raise_for_status()
#None
#一切都挺和谐哈。

HTTPError: 404 Client Error: NOT FOUND for url: http://httpbin.org/status/404

## 响应头
我们可以查看以一个 Python 字典形式展示的服务器响应头：

In [29]:
r.headers

{'Connection': 'keep-alive', 'Server': 'meinheld/0.6.1', 'Date': 'Sat, 07 Apr 2018 02:20:34 GMT', 'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'X-Powered-By': 'Flask', 'X-Processed-Time': '0', 'Content-Length': '267', 'Via': '1.1 vegur'}

但是这个字典比较特殊：它是仅为 HTTP 头部而生的。根据 RFC 2616， HTTP 头部是大小写不敏感的。

因此，我们可以使用任意大写形式来访问这些响应头字段：
```python3

r.headers['Content-Type']
'application/json'

r.headers.get('content-type')
'application/json'
```

它还有一个特殊点，那就是服务器可以多次接受同一 header，每次都使用不同的值。但 Requests 会将它们合并，这样它们就可以用一个映射来表示出来，参见 RFC 7230:

A recipient MAY combine multiple header fields with the same field name into one "field-name: field-value" pair, without changing the semantics of the message, by appending each subsequent field value to the combined field value in order, separated by a comma.

接收者可以合并多个相同名称的 header 栏位，把它们合为一个 "field-name: field-value" 配对，将每个后续的栏位值依次追加到合并的栏位值中，用逗号隔开即可，这样做不会改变信息的语义。

## Cookie
如果某个响应中包含一些 cookie，你可以快速访问它们：

In [31]:
url = 'http://example.com/some/cookie/setting/url'

r = requests.get(url)

r.cookies

#要想发送你的cookies到服务器，可以使用 cookies 参数：
url = 'http://httpbin.org/cookies'
cookies = dict(cookies_are='working')
r = requests.get(url, cookies=cookies)
r.text

<RequestsCookieJar[]>